In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [14]:
seq_df = pd.read_csv('../data/Brown_MPRA/Table 2.csv')
seq_df = seq_df.rename(columns={'Unnamed: 0': 'idx'})
seq_df

,idx,seq_ID,enhancer_ID,enhancer_seq,barcode,full_oligo_seq,source,subSource,dnaDetect,rnaDetect
0,1,CS6000114723_mm10_chr6_1,CS6000114723_mm10_chr6,TCCCCATTCCAGTGATGCTTGTCAGCCGTGCTGTGGCTAACACAGT...,ATTCCTTTAGCTACAA,GCTAAGGGCCTAACTGGCCGCTTCACTGTCCCCATTCCAGTGATGC...,MEW,MEW,False,False
1,2,CS6000114723_taeGut2_chr12_1,CS6000114723_taeGut2_chr12,CTCCATCACGGCGAATTCCTGTCAGCTGCACTCTCGCTAACACAAT...,GCCTATTAACTCACTA,GCTAAGGGCCTAACTGGCCGCTTCACTGCTCCATCACGGCGAATTC...,MEW,MEW,False,False
2,3,CS6000114723_hg38_chr3_1,CS6000114723_hg38_chr3,CTCCATCACAGTGGACGCCTGTCAGCTGCACTCTTGCTAACACAAT...,ATATGTTTTATTTCCC,GCTAAGGGCCTAACTGGCCGCTTCACTGCTCCATCACAGTGGACGC...,MEW,MEW,False,False
3,4,CS6000114723_panTro4_chr3_1,CS6000114723_panTro4_chr3,CTCCATCACAGTGGACGCCTGTCAGCTGCACTCTTGCTAACACAAT...,ACGTAAATGGATATGA,GCTAAGGGCCTAACTGGCCGCTTCACTGCTCCATCACAGTGGACGC...,MEW,MEW,False,False
4,5,CS6000114723_Raegyp2_NW_015493843.1_1,CS6000114723_Raegyp2_NW_015493843.1,CTCCATCACAGTAGATTCTTGTCAGCTGCACTCTTGCTAACACAAT...,TCGTTGGGCTAGCTGC,GCTAAGGGCCTAACTGGCCGCTTCACTGCTCCATCACAGTAGATTC...,MEW,MEW,False,True
...,...,...,...,...,...,...,...,...,...,...
9215,9216,GC70_06_20,GC70_06,CACGGAGCCGGCGGGCTAACCACGAGCGTCAGCCGGCGCCGAGGGG...,TGGACCTGGTGTAAAC,GCTAAGGGCCTAACTGGCCGCTTCACTGCACGGAGCCGGCGGGCTA...,GC,GC_70,True,True
9216,9217,GC70_07_20,GC70_07,TGACGGGATGGGACGTGGTGCAGAACGCCAATAAATCGTGGCATGG...,TGCGATGTGGCGGCGG,GCTAAGGGCCTAACTGGCCGCTTCACTGTGACGGGATGGGACGTGG...,GC,GC_70,True,True
9217,9218,GC70_08_20,GC70_08,CCAGTCCCGCGCAGGCGCTCCCGCCCGGCACGTAGGCCTCTCGCTA...,AGATGGTCTACGTGAC,GCTAAGGGCCTAACTGGCCGCTTCACTGCCAGTCCCGCGCAGGCGC...,GC,GC_70,True,True
9218,9219,GC70_09_20,GC70_09,CTAGCGGCCGGGACCCTAGCATCTCGCTCCAGAGATCTAACGGCCC...,TGGTAAAGCACAACTG,GCTAAGGGCCTAACTGGCCGCTTCACTGCTAGCGGCCGGGACCCTA...,GC,GC_70,False,False


In [15]:
seq_df['enhancer_seq'].apply(len).value_counts()

enhancer_seq
120    9220
Name: count, dtype: int64

In [16]:
seq_df['full_oligo_seq'].apply(len).value_counts()

full_oligo_seq
226    9220
Name: count, dtype: int64

In [17]:
dna_counts_df = pd.read_csv('../data/Brown_MPRA/Table 4.csv')
dna_counts = dna_counts_df.drop(columns=['Unnamed: 0'])

In [18]:
rna_counts_df = pd.read_csv('../data/Brown_MPRA/Table 5.csv')
rna_counts = rna_counts_df.drop(columns=['Unnamed: 0'])

In [19]:
common_cols = sorted(set(dna_counts.columns) & set(rna_counts.columns))
only_dna = sorted(set(dna_counts.columns) - set(rna_counts.columns))
only_rna = sorted(set(rna_counts.columns) - set(dna_counts.columns))
print("DNA 和 RNA 共同存在：", common_cols)
print("仅 DNA 存在：", only_dna)
print("仅 RNA 存在：", only_rna)

DNA 和 RNA 共同存在： ['Cortex_1_1', 'Cortex_2_1', 'Cortex_6_1', 'Cortex_7_1', 'Cortex_8_1', 'HMC3_C_2', 'HMC3_E_2', 'Hippocampus_1_1', 'Hippocampus_2_1', 'Hippocampus_5_1', 'Hippocampus_6_1', 'Hippocampus_7_1', 'Hippocampus_8_1', 'Hypothalamus_10_2', 'Hypothalamus_9_2', 'Kidney_10_2', 'Kidney_9_2', 'Liver_10_2', 'Liver_7_1', 'Liver_8_1', 'Liver_9_2', 'Lung_10_2', 'M1_1_1', 'M1_2_1', 'M1_5_1', 'M1_6_1', 'M1_7_1', 'M1_8_1', 'Muscle_10_2', 'PFC_5_1', 'PFC_6_1', 'PFC_7_1', 'PFC_8_1', 'Striatum_1_1', 'Striatum_2_1', 'Striatum_5_1', 'Striatum_6_1', 'Striatum_7_1', 'Striatum_8_1']
仅 DNA 存在： ['HMC3_D_2', 'Heart_9_2', 'Ovaries_10_2', 'Testies_9_2']
仅 RNA 存在： ['Cortex_1_2', 'Cortex_2_2', 'Cortex_8_2', 'HMC3_A_2', 'HMC3_B_2', 'Hippocampus_2_2', 'Hippocampus_5_2', 'Liver_7_2', 'Liver_8_2', 'Striatum_1_2', 'Striatum_5_2']


In [20]:
dna_counts = dna_counts[common_cols]
rna_counts = rna_counts[common_cols]

In [21]:
dna_cpm = (dna_counts / dna_counts.sum()) * 1e6
rna_cpm = (rna_counts / rna_counts.sum()) * 1e6
rna_cpm

,Cortex_1_1,Cortex_2_1,Cortex_6_1,Cortex_7_1,Cortex_8_1,HMC3_C_2,HMC3_E_2,Hippocampus_1_1,Hippocampus_2_1,Hippocampus_5_1,Hippocampus_6_1,Hippocampus_7_1,Hippocampus_8_1,Hypothalamus_10_2,Hypothalamus_9_2,Kidney_10_2,Kidney_9_2,Liver_10_2,Liver_7_1,Liver_8_1,Liver_9_2,Lung_10_2,M1_1_1,M1_2_1,M1_5_1,M1_6_1,M1_7_1,M1_8_1,Muscle_10_2,PFC_5_1,PFC_6_1,PFC_7_1,PFC_8_1,Striatum_1_1,Striatum_2_1,Striatum_5_1,Striatum_6_1,Striatum_7_1,Striatum_8_1
0,304.853,115.127,202.016,141.455,522.195,360.052,188.508,365.619,289.184,81.415,63.319,199.151,948.389,1.171,10.970,8.187,1.481,91.917,302.585,14.991,5.254,69.368,1064.409,669.876,111.881,1030.204,283.785,338.551,1.147,959.461,71.265,67.951,345.418,283.134,472.413,124.116,28.990,1243.770,1014.997
1,327.569,102.187,572.461,369.206,714.072,268.661,244.186,118.098,295.826,45.254,15.755,459.807,231.194,0.473,12.163,0.000,0.740,56.358,189.654,8.072,5.993,43.008,369.000,7.297,34.920,12.924,9.426,18.933,0.765,1.160,20.236,21.812,5.416,298.506,22.336,58.422,139.880,492.342,348.236
2,78.943,29.860,3.621,6.072,8.184,67.363,53.066,1.699,3.361,125.824,4.175,7.540,137.673,0.045,0.954,0.000,0.000,61.054,17.128,1721.021,4.115,6.937,8.453,206.949,33.564,10.818,536.599,14.079,0.000,4.289,65.547,51.173,1193.869,2.778,521.473,19.056,164.222,10.366,9.470
3,184.038,43.131,83.715,200.938,9.613,134.740,164.387,95.843,78.988,1270.081,6.411,13.294,5.794,0.158,5.485,0.000,1.110,24.153,4.470,1.301,2.936,54.107,7.887,3.991,28.140,313.436,7.936,4.854,0.382,3.078,29.914,31.040,393.811,286.432,18.268,62.685,36.766,902.207,12.733
4,75.190,35.168,154.571,204.296,328.456,45.357,83.016,80.129,137.650,11.631,4.225,6.253,7.551,0.090,2.862,0.000,0.000,27.508,310.608,6.268,3.116,13.874,4.894,3.991,12.883,6.701,3.782,154.973,0.000,0.807,6.599,7.215,2.959,168.416,8.775,30.841,8.114,98.782,338.177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,371.025,556.391,1111.274,885.240,260.340,314.025,566.455,249.843,128.806,1136.010,28.826,1229.954,206.400,0.968,9.539,3.509,0.740,119.760,426.151,10.201,9.608,3698.699,140.473,10.946,64.416,21.253,21.316,237.003,1.147,3.481,30.354,39.261,5.566,10.205,868.404,68.954,14.495,259.450,14.377
3979,260.081,438.942,146.642,1031.745,298.706,689.843,365.686,767.679,912.168,186.093,61.032,827.212,1529.861,1.126,20.032,8.187,1.481,114.393,12.917,32.052,9.069,184.519,228.082,387.217,151.887,54.282,622.007,388.017,1.530,729.838,130.653,94.461,527.555,854.259,892.655,2695.206,2343.356,985.833,361.337
3980,75.409,31.851,98.885,3.923,5.066,72.708,120.494,78.572,113.841,21.993,6.163,7.672,371.915,0.113,3.816,0.000,0.370,46.629,389.119,4.731,3.396,22.198,11.770,1.938,4.407,2.585,1.289,2.306,0.000,3.381,49.270,1248.796,627.499,2.181,523.468,10.782,3.169,3.698,4.073
3981,14.892,18.911,4.432,5.615,326.377,56.409,98.183,20.499,1.481,4.864,136.677,4.519,5.574,0.113,1.669,1.170,0.740,23.147,1.294,2.750,1.318,5.549,202.843,5.131,22.037,5.457,226.541,9.260,0.000,0.505,6.599,5.872,2.307,3.479,4.068,9.779,3.803,5.743,234.202


In [22]:
pseudo = 1
log_ratio = np.log2((rna_cpm + pseudo) / (dna_cpm + pseudo))
log_ratio

,Cortex_1_1,Cortex_2_1,Cortex_6_1,Cortex_7_1,Cortex_8_1,HMC3_C_2,HMC3_E_2,Hippocampus_1_1,Hippocampus_2_1,Hippocampus_5_1,Hippocampus_6_1,Hippocampus_7_1,Hippocampus_8_1,Hypothalamus_10_2,Hypothalamus_9_2,Kidney_10_2,Kidney_9_2,Liver_10_2,Liver_7_1,Liver_8_1,Liver_9_2,Lung_10_2,M1_1_1,M1_2_1,M1_5_1,M1_6_1,M1_7_1,M1_8_1,Muscle_10_2,PFC_5_1,PFC_6_1,PFC_7_1,PFC_8_1,Striatum_1_1,Striatum_2_1,Striatum_5_1,Striatum_6_1,Striatum_7_1,Striatum_8_1
0,-0.125,-1.622,-0.666,-1.493,0.774,0.629,0.815,0.184,-0.292,-2.100,-2.581,-1.330,0.912,-6.872,-4.125,-5.648,-6.651,-2.315,-0.481,-4.349,-6.024,-4.306,1.631,0.936,-1.707,1.578,-0.511,0.424,-7.696,1.379,-0.918,-2.418,0.582,-0.208,0.298,-1.317,-3.465,1.586,1.709
1,0.609,-0.800,1.752,0.468,1.501,0.177,-0.103,-0.852,0.379,-2.134,-4.403,1.286,-0.520,-7.763,-5.702,-6.625,-7.612,0.089,0.269,-4.419,-5.352,-2.935,0.583,-4.682,-3.258,-4.744,-3.848,-3.983,0.448,-5.919,-2.492,-3.164,-5.346,0.640,-3.138,-1.249,-0.544,0.945,0.529
2,0.656,-1.041,-3.900,-1.117,-1.541,0.421,0.524,-4.033,-3.136,0.207,-4.539,-4.054,0.865,-7.420,-0.015,-4.783,-4.737,-0.141,-2.281,4.600,-3.106,0.686,-1.014,1.661,-0.924,-1.482,3.024,-2.566,-0.102,-5.014,-0.662,2.093,5.087,-4.269,3.409,-1.081,0.720,-1.258,-1.919
3,0.203,-1.990,-0.708,0.112,-3.431,-0.246,-0.944,-0.721,-0.425,3.389,-4.265,-3.601,-5.290,-6.521,-2.590,-6.045,0.028,-1.265,-4.921,-5.995,-3.506,4.538,-3.858,-5.209,-1.165,2.010,-4.789,-5.362,0.467,-4.619,1.259,-3.350,2.019,0.837,-2.885,-0.113,-2.103,1.984,-3.302
4,-0.040,-1.338,0.557,1.201,1.923,-0.842,0.110,-0.275,0.767,-3.167,-4.818,-2.824,-2.156,-7.324,-4.738,-6.411,-5.557,-0.966,1.946,-1.570,-3.435,0.097,-4.141,-3.224,-2.811,-4.010,-4.231,1.466,0.000,-2.818,-4.171,-3.825,-4.667,0.813,-3.231,-1.930,-3.004,0.385,2.144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,-0.490,0.095,1.436,1.554,-1.015,-0.751,0.287,-1.297,-2.184,1.009,-4.278,1.272,-1.285,-8.320,-4.639,-7.273,-8.496,-2.230,-0.663,-6.214,-5.974,3.565,-1.838,-5.693,-2.715,-4.449,-4.854,-1.284,-7.389,-7.109,-3.961,-4.091,-5.490,-5.366,1.044,-3.048,-4.731,-1.238,-4.774
3979,-1.616,-0.924,-2.756,0.415,-1.405,0.380,2.089,-0.206,-0.037,-1.977,-3.857,-0.048,0.598,-7.880,-5.342,-6.640,-8.619,-3.068,-5.786,-4.945,-6.727,-2.847,-2.012,-1.242,-2.314,-3.575,-0.647,-1.585,-8.984,-0.315,-2.453,-3.472,-0.647,-0.064,0.188,1.620,1.190,0.384,-1.137
3980,-0.051,-1.777,0.285,-2.765,-3.013,-0.428,-0.478,-0.349,0.298,-1.648,-3.711,-3.784,2.452,-0.235,2.071,-3.925,-0.110,-2.217,2.069,0.788,-3.888,-2.480,-2.266,-4.916,-4.067,-5.501,-3.540,-2.624,-0.287,-2.899,4.073,3.402,5.056,-4.796,2.929,-2.522,-4.482,-2.624,-3.998
3981,-2.087,-1.795,-4.308,-3.912,2.832,-0.540,0.515,-0.949,-4.528,-3.456,1.375,-4.411,-4.030,-6.215,0.892,-6.796,-6.917,-1.622,-4.934,0.176,-4.705,-5.306,1.609,-3.659,-0.496,-3.691,4.698,-3.319,0.000,-4.257,1.155,0.005,-5.287,-3.985,-2.851,-2.833,-3.825,-3.235,1.624


In [23]:
barcode_df = dna_counts_df[['Unnamed: 0']]
barcode_df = barcode_df.rename(columns={'Unnamed: 0': 'barcode'})
barcode_df

,barcode
0,CTCCCGCAGTCCGCTG
1,CTACAGGACATACGTT
2,TCAACTGTGCTCTAAC
3,CGTCAAGAGGTAGTGG
4,TTTGAGTCGTACAAGG
...,...
3978,AGTATTTCGGGGTATC
3979,TATGGCGATCTCTGCG
3980,TGGACCTGGTGTAAAC
3981,TGCGATGTGGCGGCGG


In [24]:
merged_df = pd.merge(left=barcode_df, right=seq_df, on="barcode", how="left")
merged_df

,barcode,idx,seq_ID,enhancer_ID,enhancer_seq,full_oligo_seq,source,subSource,dnaDetect,rnaDetect
0,CTCCCGCAGTCCGCTG,10,CS6000114687_hg38_chr3_1,CS6000114687_hg38_chr3,TAGAGGGCACTGTGGTATTTAGGCCACTTTGACCTCTTCGAGTACT...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAGAGGGCACTGTGGTAT...,MEW,MEW,True,True
1,CTACAGGACATACGTT,15,CS6000115061_rheMac8_chr2_1,CS6000115061_rheMac8_chr2,TAATTTGTCATGCACACTCTGATTGCTTGGCATATTGGCAGAGTCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAATTTGTCATGCACACT...,MEW,MEW,True,True
2,TCAACTGTGCTCTAAC,16,CS6000115061_taeGut2_chr1_1,CS6000115061_taeGut2_chr1,TAAATCCTGGTGCATTCTCTGGGGTTTGCAGGATCCTGGGCTTCCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAAATCCTGGTGCATTCT...,MEW,MEW,True,True
3,CGTCAAGAGGTAGTGG,17,CS6000115061_hg38_chr3_1,CS6000115061_hg38_chr3,TAATTTGTCATGCACACTCTGATTGCTTGGCATATTGGCAGAGTCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAATTTGTCATGCACACT...,MEW,MEW,True,True
4,TTTGAGTCGTACAAGG,19,CS6000115061_myoLuc2_GL429887_1,CS6000115061_myoLuc2_GL429887,TAATTTGTCATGCACACTCTGATTGCTAGGCATATTGGCAGAGTCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAATTTGTCATGCACACT...,MEW,MEW,True,True
...,...,...,...,...,...,...,...,...,...,...
3978,AGTATTTCGGGGTATC,9208,GC50_08_20,GC50_08,TCAATAGCAGACTGTCCTCGAATCTGGAAGACATCTCAGTAGGCTC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTCAATAGCAGACTGTCCT...,GC,GC_50,True,True
3979,TATGGCGATCTCTGCG,9210,GC50_10_20,GC50_10,GTGGTAATGGCCTACGCTTTCGATACTCGGGGCACCTCACTCAGCT...,GCTAAGGGCCTAACTGGCCGCTTCACTGGTGGTAATGGCCTACGCT...,GC,GC_50,True,True
3980,TGGACCTGGTGTAAAC,9216,GC70_06_20,GC70_06,CACGGAGCCGGCGGGCTAACCACGAGCGTCAGCCGGCGCCGAGGGG...,GCTAAGGGCCTAACTGGCCGCTTCACTGCACGGAGCCGGCGGGCTA...,GC,GC_70,True,True
3981,TGCGATGTGGCGGCGG,9217,GC70_07_20,GC70_07,TGACGGGATGGGACGTGGTGCAGAACGCCAATAAATCGTGGCATGG...,GCTAAGGGCCTAACTGGCCGCTTCACTGTGACGGGATGGGACGTGG...,GC,GC_70,True,True


In [26]:
final_df = pd.concat([merged_df[['barcode', 'enhancer_seq', 'full_oligo_seq']], log_ratio], axis=1)
final_df

,barcode,enhancer_seq,full_oligo_seq,Cortex_1_1,Cortex_2_1,Cortex_6_1,Cortex_7_1,Cortex_8_1,HMC3_C_2,HMC3_E_2,Hippocampus_1_1,Hippocampus_2_1,Hippocampus_5_1,Hippocampus_6_1,Hippocampus_7_1,Hippocampus_8_1,Hypothalamus_10_2,Hypothalamus_9_2,Kidney_10_2,Kidney_9_2,Liver_10_2,Liver_7_1,Liver_8_1,Liver_9_2,Lung_10_2,M1_1_1,M1_2_1,M1_5_1,M1_6_1,M1_7_1,M1_8_1,Muscle_10_2,PFC_5_1,PFC_6_1,PFC_7_1,PFC_8_1,Striatum_1_1,Striatum_2_1,Striatum_5_1,Striatum_6_1,Striatum_7_1,Striatum_8_1
0,CTCCCGCAGTCCGCTG,TAGAGGGCACTGTGGTATTTAGGCCACTTTGACCTCTTCGAGTACT...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAGAGGGCACTGTGGTAT...,-0.125,-1.622,-0.666,-1.493,0.774,0.629,0.815,0.184,-0.292,-2.100,-2.581,-1.330,0.912,-6.872,-4.125,-5.648,-6.651,-2.315,-0.481,-4.349,-6.024,-4.306,1.631,0.936,-1.707,1.578,-0.511,0.424,-7.696,1.379,-0.918,-2.418,0.582,-0.208,0.298,-1.317,-3.465,1.586,1.709
1,CTACAGGACATACGTT,TAATTTGTCATGCACACTCTGATTGCTTGGCATATTGGCAGAGTCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAATTTGTCATGCACACT...,0.609,-0.800,1.752,0.468,1.501,0.177,-0.103,-0.852,0.379,-2.134,-4.403,1.286,-0.520,-7.763,-5.702,-6.625,-7.612,0.089,0.269,-4.419,-5.352,-2.935,0.583,-4.682,-3.258,-4.744,-3.848,-3.983,0.448,-5.919,-2.492,-3.164,-5.346,0.640,-3.138,-1.249,-0.544,0.945,0.529
2,TCAACTGTGCTCTAAC,TAAATCCTGGTGCATTCTCTGGGGTTTGCAGGATCCTGGGCTTCCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAAATCCTGGTGCATTCT...,0.656,-1.041,-3.900,-1.117,-1.541,0.421,0.524,-4.033,-3.136,0.207,-4.539,-4.054,0.865,-7.420,-0.015,-4.783,-4.737,-0.141,-2.281,4.600,-3.106,0.686,-1.014,1.661,-0.924,-1.482,3.024,-2.566,-0.102,-5.014,-0.662,2.093,5.087,-4.269,3.409,-1.081,0.720,-1.258,-1.919
3,CGTCAAGAGGTAGTGG,TAATTTGTCATGCACACTCTGATTGCTTGGCATATTGGCAGAGTCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAATTTGTCATGCACACT...,0.203,-1.990,-0.708,0.112,-3.431,-0.246,-0.944,-0.721,-0.425,3.389,-4.265,-3.601,-5.290,-6.521,-2.590,-6.045,0.028,-1.265,-4.921,-5.995,-3.506,4.538,-3.858,-5.209,-1.165,2.010,-4.789,-5.362,0.467,-4.619,1.259,-3.350,2.019,0.837,-2.885,-0.113,-2.103,1.984,-3.302
4,TTTGAGTCGTACAAGG,TAATTTGTCATGCACACTCTGATTGCTAGGCATATTGGCAGAGTCC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTAATTTGTCATGCACACT...,-0.040,-1.338,0.557,1.201,1.923,-0.842,0.110,-0.275,0.767,-3.167,-4.818,-2.824,-2.156,-7.324,-4.738,-6.411,-5.557,-0.966,1.946,-1.570,-3.435,0.097,-4.141,-3.224,-2.811,-4.010,-4.231,1.466,0.000,-2.818,-4.171,-3.825,-4.667,0.813,-3.231,-1.930,-3.004,0.385,2.144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,AGTATTTCGGGGTATC,TCAATAGCAGACTGTCCTCGAATCTGGAAGACATCTCAGTAGGCTC...,GCTAAGGGCCTAACTGGCCGCTTCACTGTCAATAGCAGACTGTCCT...,-0.490,0.095,1.436,1.554,-1.015,-0.751,0.287,-1.297,-2.184,1.009,-4.278,1.272,-1.285,-8.320,-4.639,-7.273,-8.496,-2.230,-0.663,-6.214,-5.974,3.565,-1.838,-5.693,-2.715,-4.449,-4.854,-1.284,-7.389,-7.109,-3.961,-4.091,-5.490,-5.366,1.044,-3.048,-4.731,-1.238,-4.774
3979,TATGGCGATCTCTGCG,GTGGTAATGGCCTACGCTTTCGATACTCGGGGCACCTCACTCAGCT...,GCTAAGGGCCTAACTGGCCGCTTCACTGGTGGTAATGGCCTACGCT...,-1.616,-0.924,-2.756,0.415,-1.405,0.380,2.089,-0.206,-0.037,-1.977,-3.857,-0.048,0.598,-7.880,-5.342,-6.640,-8.619,-3.068,-5.786,-4.945,-6.727,-2.847,-2.012,-1.242,-2.314,-3.575,-0.647,-1.585,-8.984,-0.315,-2.453,-3.472,-0.647,-0.064,0.188,1.620,1.190,0.384,-1.137
3980,TGGACCTGGTGTAAAC,CACGGAGCCGGCGGGCTAACCACGAGCGTCAGCCGGCGCCGAGGGG...,GCTAAGGGCCTAACTGGCCGCTTCACTGCACGGAGCCGGCGGGCTA...,-0.051,-1.777,0.285,-2.765,-3.013,-0.428,-0.478,-0.349,0.298,-1.648,-3.711,-3.784,2.452,-0.235,2.071,-3.925,-0.110,-2.217,2.069,0.788,-3.888,-2.480,-2.266,-4.916,-4.067,-5.501,-3.540,-2.624,-0.287,-2.899,4.073,3.402,5.056,-4.796,2.929,-2.522,-4.482,-2.624,-3.998
3981,TGCGATGTGGCGGCGG,TGACGGGATGGGACGTGGTGCAGAACGCCAATAAATCGTGGCATGG...,GCTAAGGGCCTAACTGGCCGCTTCACTGTGACGGGATGGGACGTGG...,-2.087,-1.795,-4.308,-3.912,2.832,-0.540,0.515,-0.949,-4.528,-3.456,1.375,-4.411,-4.030,-6.215,0.892,-6.796,-6.917,-1.622,-4.934,0.176,-4.705,-5.306,1.609,-3.659,-0.496,-3.

In [27]:
final_df.to_csv('../data/Brown_MPRA/Brown_MPRA_final.csv', index=False)